In [1]:

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from glob import glob
from keras import backend as K
sys.path.append("../NEAT")
from tqdm import tqdm
import cv2
from keras.models import Model, load_model
from NEATModels import yolo_loss, Concat
from stardist.models import StarDist2D
from csbdeep.models import Config, CARE
from csbdeep.utils import normalize
from NEATUtils import helpers, NMS, MultiModelPrediction
from NEATUtils.helpers import smallzero_pad, WatershedwithMask, fill_label_holes,MaxProjectDist, multiplot,Integer_to_border
import glob
from skimage.filters import threshold_otsu
from scipy.ndimage.morphology import binary_fill_holes
import numpy as np
from skimage.morphology import remove_small_objects
from tifffile import imread, imwrite
try:
    from pathlib import Path
    Path().expanduser()
except (ImportError,AttributeError):
    from pathlib2 import Path
try:
    import tempfile
    tempfile.TemporaryDirectory
except (ImportError,AttributeError):
    from backports import tempfile

Using TensorFlow backend.


In [2]:
#Path to your model directory
ModelDirectory = '/home/sancere/VarunNewton/CurieDeepLearningModels/O-NEATweights/'
#Path to timelapse image directory
MovieDirectory =  '//home/sancere/VarunCom/commun/Maria_to_Varun/Pcna4b/Rotated/'
ResultsDirectory = MovieDirectory +  '/VarunResults/'


SegmentationMaskModel = 'DrosMaskBin1'
SegmentationStarModel = 'LucasDrosSmartSeeds'



In [3]:

model = StarDist2D(config = None, name = SegmentationStarModel, basedir = ModelDirectory)
UnetModel = CARE(config = None, name = SegmentationMaskModel, basedir = ModelDirectory)


Loading network weights from 'weights_best.h5'.
Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.
Loading network weights from 'weights_best.h5'.


In [4]:
Path(ResultsDirectory).mkdir(exist_ok = True)
Path(ResultsDirectory + '/Mask/').mkdir(exist_ok = True)
Raw_path = os.path.join(MovieDirectory, '*tif')
X = glob.glob(Raw_path)
axes = 'TYX'
axis_norm = (0,1)

for fname in X:
    image = imread(fname)
    
    
    x = image
    originalX = x.shape[0]
    originalY = x.shape[1]  

    #Get Name
    Name = os.path.basename(os.path.splitext(fname)[0])

    #UNET Prediction

    Segmented = UnetModel.predict(x,'YX')
    thresh = threshold_otsu(Segmented) 
    Binary = Segmented > thresh
    Finalimage = Binary


    #Stardist Prediction

    x = normalize(x,1,99.8,axis=axis_norm)
    x = smallzero_pad(x, 64, 64)

    MidImage, details = model.predict_instances(x)
    StarImage = MidImage[:originalX, :originalY]

    smallprob, smalldist = model.predict(x)
    grid = model.config.grid
    midprob = cv2.resize(smallprob, dsize=(smallprob.shape[1] * grid[1] , smallprob.shape[0] * grid[0] ))
    middist = cv2.resize(smalldist, dsize=(smalldist.shape[1] * grid[1] , smalldist.shape[0] * grid[0] ))
    dist = MaxProjectDist(middist)
    prob = dist[:originalX, :originalY]
    Watershed, Markers = WatershedwithMask(prob,StarImage,Finalimage,  model.config.grid)

    Watershed = Integer_to_border(Watershed)
              
            #Save different method segmentation
                   unhashable type: 'numpy.ndarray' list
    imwrite((ResultsDirectory  + Name+ '.tif' ) , Watershed.astype('uint8'))
    imwrite((ResultsDirectory+ '/Mask/'  + Name+ '.tif' ) , Finalimage.astype('uint8'))
        
    #filelist = glob.glob(ResultsDirectory + '*' +  (os.path.splitext(os.path.basename(fname))[0] + '.csv'))
    #for filepath in filelist:
      #os.remove(filepath)
    
    
    #print("Obtaining Dynamic events")
    #MultiModelPrediction.SegDynamicEvents(image,Watershed, fname, CubicModels, Categories_Name,
                            #Mode,n_tiles, ResultsDirectory,TrainshapeX, TrainshapeY, TimeFrames)
    #print("Obtaining Static events")
    #MultiModelPrediction.StaticEvents(image, fname, StaticModels, StaticCategories_Name, StaticCutoffDistance,
                           #n_tiles, ResultsDirectory,TrainshapeX, TrainshapeY) 
    

    
    
